In [1]:
import pandas as pd
pd.set_option("display.max_columns",250)
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
import yaml as yaml

sns.set()

In [2]:
with open(r'N:/Projects/CMAP_Activitysim/cmap_abm_lf/survey_data_prep/cmap_inputs.yml') as file:
    settings = yaml.full_load(file)

output_path = r"output"

processed_transit_survey_path = r"output"
spa_input_path = settings['SPA_input_dir']
spa_output_path = settings['SPA_output_dir'] 

tour_mode_choice_output_path = os.path.join(output_path, 'tour_mode_choice')
trip_mode_choice_output_path = os.path.join(output_path, 'trip_mode_choice')

cmap_weights_path = os.path.join(settings['popsim_folder'], 'output', 'final_summary_hh_weights.csv')
nirpc_weights_path = os.path.join(settings['popsim_folder_nirpc'],  'output', 'final_summary_hh_weights.csv')

trip_weights_path = os.path.join(settings['proj_dir'],'underreporting_correction', 'trip_weights.csv')

In [3]:
# Reading in tables
obs_df = pd.read_csv(os.path.join(processed_transit_survey_path, 'processed_obs.csv'))

spa_in_hh_df = pd.read_csv(os.path.join(spa_input_path, "HH_SPA_INPUT.csv"), encoding='latin1')
spa_in_per_df = pd.read_csv(os.path.join(spa_input_path, "PER_SPA_INPUT.csv"), encoding='latin1')
spa_in_place_df = pd.read_csv(os.path.join(spa_input_path, "PLACE_SPA_INPUT.csv"), encoding='latin1')

spa_out_per_df = pd.read_csv(os.path.join(spa_output_path, "persons.csv"), encoding='latin1')
spa_out_tours_df = pd.read_csv(os.path.join(spa_output_path, "tours.csv"), encoding='latin1')
spa_out_trips_df = pd.read_csv(os.path.join(spa_output_path, "trips.csv"), encoding='latin1')

cmap_weights = pd.read_csv(cmap_weights_path)
nirpc_weights = pd.read_csv(nirpc_weights_path)
trip_weights = pd.read_csv(trip_weights_path)
# warnings on column types...
spa_out_per_df.head()

C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (1,2,3,4,7,8,9,10,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,35,36,38,39,44,49,62,81,82,83,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,HH_ID,PER_ID,PERSONTYPE,AGE_CAT,EMPLY,HOURS_CAT,EMP_CAT,STUDE,SCHOL,STU_CAT,PERSONTYPE0,EMP_CAT0,STU_CAT0,ERROR
0,20000083,1,1,6,1,NaN,1,0,-9,3,NaN,NaN,NaN,NaN
1,20000136,1,2,5,2,NaN,2,0,-9,3,NaN,NaN,NaN,NaN
2,20000136,2,1,5,1,NaN,1,0,-9,3,NaN,NaN,NaN,NaN
3,20000136,3,8,1,3,NaN,4,0,1,3,NaN,NaN,NaN,NaN
4,20000228,1,2,6,2,NaN,2,0,-9,3,NaN,NaN,NaN,NaN


In [4]:
output_excel_path = os.path.join(output_path, 'calibration_target_tables.xlsx')
excel_writer = pd.ExcelWriter(output_excel_path, engine='xlsxwriter')

# Matching mode  and purpose names between surveys

In [5]:
output_calibration_modes = ['DRIVEALONE','SHARED2','SHARED3', 'WALK', 'BIKE', 'WALK-TRANSIT',
                            'PNR-TRANSIT','KNR-TRANSIT', 'TNC-TRANSIT', 'TAXI', 'TNC-REG', 'TNC-SHARED','SCHOOLBUS', 'All']
output_calibration_purposes = ['Work', 'University', 'School', 'Work sub-tour', 'Ind-Maintenance', 'Ind-Discretionary',
                               'Joint-Maintenance', 'Joint-Discretionary',  'All']

#### Household Travel Survey

In [6]:
weights = pd.concat([nirpc_weights, cmap_weights])
spa_out_trips_df = pd.merge(spa_out_trips_df, weights, left_on  = 'HH_ID', right_on = 'sampno')
# expansion factor already includes the weight
spa_out_trips_df["PER_WEIGHT"] = spa_out_trips_df['SUBREGCluster_balanced_weight']


In [7]:
adults_df = spa_in_per_df[spa_in_per_df['EMPLY'].isin(range(1, 2 + 1))]['SAMPN'].value_counts().reset_index()
adults_df.columns=['SAMPN', 'num_workers']

print(adults_df.groupby('num_workers').count())


             SAMPN
num_workers       
1             5764
2             5493
3              627
4              140
5               17
6                2
7                2


In [8]:
# Calculating auto sufficiency for households

spa_in_hh_df.drop(columns=['num_workers'], errors = 'ignore')
spa_in_hh_df = pd.merge(
    spa_in_hh_df,
    adults_df[['SAMPN', 'num_workers']],
    how='left',
    left_on=['SAMPN'],
    right_on=['SAMPN']
)



In [9]:
spa_in_hh_df['auto_suff'] = 0

spa_in_hh_df.loc[pd.isnull(spa_in_hh_df.num_workers), 'num_workers']=0
spa_in_hh_df.loc[((spa_in_hh_df['HH_VEH'] < spa_in_hh_df['num_workers']) 
                     & (spa_in_hh_df['HH_VEH'] > 0)),
                    'auto_suff'] = 1
spa_in_hh_df.loc[((spa_in_hh_df['HH_VEH'] >= spa_in_hh_df['num_workers']) 
                     & (spa_in_hh_df['HH_VEH'] > 0)),
                    'auto_suff'] = 2



In [10]:
spa_in_hh_df[['SAMPN', 'auto_suff']]

,SAMPN,auto_suff
0,20000083,2
1,20000136,1
2,20000228,2
3,20000248,2
4,20000300,0
...,...,...
13984,50074311,2
13985,50074376,1
13986,50074386,2
13987,50074395,2


In [11]:
# Merging auto suffiency from Households
spa_out_trips_df = pd.merge(
    spa_out_trips_df,
    spa_in_hh_df[['SAMPN', 'auto_suff']],
    how='left',
    left_on=['HH_ID'],
    right_on=['SAMPN']
)

spa_out_tours_df = pd.merge(
    spa_out_tours_df,
    spa_in_hh_df[['SAMPN', 'auto_suff']],
    how='left',
    left_on=['HH_ID'],
    right_on=['SAMPN']
)



# # Merging weighting variables for tours
spa_out_trips_df = pd.merge(
    spa_out_trips_df,
    trip_weights,
    how='left',
    left_on=['HH_ID', 'PER_ID', 'DEST_PLACENO'],
    right_on=['sampno', 'perno', 'PLANO']
)




In [12]:
# get tour weight

tour_weights = spa_out_trips_df.groupby(['HH_ID', 'PER_ID', 'TOUR_ID'])['trip_weight'].mean().reset_index()

spa_out_tours_df = pd.merge(
    spa_out_tours_df,
    tour_weights,
    how='left',
    on=['HH_ID', 'PER_ID', 'TOUR_ID']
)


spa_out_tours_df["HH_WEIGHT"] = spa_out_tours_df['trip_weight']
spa_out_tours_df["PER_WEIGHT"] = spa_out_tours_df['trip_weight']


In [13]:

spa_out_trips_df["HH_WEIGHT"] = spa_out_trips_df['trip_weight']
spa_out_trips_df["PER_WEIGHT"] = spa_out_trips_df['trip_weight']


In [14]:
# Merging joint status and subtour status from tours to trips
spa_out_trips_df = pd.merge(
    spa_out_trips_df,
    spa_out_tours_df[['HH_ID', 'PER_ID', 'TOUR_ID', 'JOINT_STATUS', 'IS_SUBTOUR', 'HH_WEIGHT']],
    how='left',
    on=['HH_ID', 'PER_ID', 'TOUR_ID']
)



#### Removing duplicate joint trips and tours

In [15]:
spa_out_tours_df.loc[spa_out_tours_df['JOINT_STATUS'] == 3, 'PER_WEIGHT'].sum()

1394025.7076469976

In [16]:
# fully joint tours are joint_status == 3
joint_tours_df = spa_out_tours_df[spa_out_tours_df['JOINT_STATUS'] == 3]
print("Unweighted number of joint tours: ", len(joint_tours_df))

# selecting the tour from the first person in the tour file
unique_joint_tours_df = joint_tours_df.groupby(['HH_ID', 'TOUR_ID']).first().reset_index()
unique_joint_tours_df['unique_joint_tour'] = 1
print("Unweighted number of unique joint tours: ", len(unique_joint_tours_df))

# denoting the joint tour to keep
spa_out_tours_df = pd.merge(
    spa_out_tours_df,
    unique_joint_tours_df[['HH_ID', 'PER_ID', 'TOUR_ID', 'unique_joint_tour']],
    how='left',
    on=['HH_ID', 'PER_ID', 'TOUR_ID']
)

# removing non-unique joint tours
spa_out_tours_df = spa_out_tours_df[
    (spa_out_tours_df['JOINT_STATUS'] != 3)
    | ((spa_out_tours_df['JOINT_STATUS'] == 3) & (spa_out_tours_df['unique_joint_tour'] == 1))]


assert len(spa_out_tours_df[spa_out_tours_df['JOINT_STATUS'] == 3]) == len(unique_joint_tours_df), "Joint tours not unique!"

Unweighted number of joint tours:  4161
Unweighted number of unique joint tours:  2403


In [17]:
# fully joint trips are joint_status == 3
joint_trips_df = spa_out_trips_df[(spa_out_trips_df['FULLY_JOINT'] == 1)]
print("Unweighted number of joint trips: ", len(joint_trips_df))

# selecting the tour from the first person in the tour file
unique_joint_trips_df = joint_trips_df.groupby(['HH_ID', 'TOUR_ID', 'TRIP_ID']).first().reset_index()
unique_joint_trips_df['unique_joint_trip'] = 1
print("Unweighted number of unique joint trips: ", len(unique_joint_trips_df))

# denoting the joint tour to keep
spa_out_trips_df = pd.merge(
    spa_out_trips_df,
    unique_joint_trips_df[['HH_ID', 'PER_ID', 'TOUR_ID', 'TRIP_ID', 'unique_joint_trip']],
    how='left',
    on=['HH_ID', 'PER_ID', 'TOUR_ID', 'TRIP_ID']
)
# removing non-unique joint trips. All joint trips should belong to a fully joint tour (checked implicity in assert)
spa_out_trips_df = spa_out_trips_df[
    (spa_out_trips_df['JOINT_STATUS'] != 3)
    | ((spa_out_trips_df['JOINT_STATUS'] == 3) & (spa_out_trips_df['unique_joint_trip'] == 1))]



Unweighted number of joint trips:  10503
Unweighted number of unique joint trips:  6062


#### Mapping spa purpose codes to calibration purpose categories

In [18]:
spa_purp_dict = {
    0: 'HOME',
    1: 'WORK',
    2: 'UNIVERSITY',
    3: 'SCHOOL',
    4: 'ESCORTING',
    5: 'SHOPPING',
    6: 'MAINTENANCE',
    7: 'EAT OUT',
    8: 'SOCIAL/VISIT',
    9: 'DISCRETIONARY',
    10: 'WORK-RELATED',
    11: 'LOOP',
    12: 'CHANGE MODE',
    13: 'OTHER',
}
spa_out_trips_df['spa_tour_purpose'] = spa_out_trips_df['TOURPURP'].apply(lambda x: spa_purp_dict[x])

spa_to_abms_purp_dict = {
     'HOME': 'Home',
     'WORK': 'Work',
     'UNIVERSITY': 'University',
     'SCHOOL': 'School',
     'ESCORTING': 'Maintenance',
     'SHOPPING': 'Maintenance',
     'MAINTENANCE': 'Maintenance',
     'EAT OUT': 'Discretionary',
     'SOCIAL/VISIT': 'Discretionary',
     'DISCRETIONARY': 'Discretionary',
     'WORK-RELATED': 'Maintenance',
     'LOOP': 'Discretionary',
     'CHANGE MODE': 'Change Mode',
     'OTHER': 'Discretionary',
}
spa_out_trips_df['tour_purpose'] = spa_out_trips_df['spa_tour_purpose'].apply(lambda x: spa_to_abms_purp_dict[x])
spa_out_trips_df.loc[spa_out_trips_df['IS_SUBTOUR'] == 1, 'tour_purpose'] = 'Work sub-tour'

In [19]:
spa_out_trips_df['tour_purpose'].value_counts(dropna=False)

Maintenance      35185
Work             34377
Discretionary    22422
School           10396
Work sub-tour     4410
University        2144
Change Mode        277
Name: tour_purpose, dtype: int64

In [20]:
def group_tour_purposes(tour_purpose):
    if tour_purpose in ['Work', 'University', 'School', 'Work sub-tour']:
        return tour_purpose
    if tour_purpose in ['Escorting','Shopping','Maintenance']:
        return 'Ind-Maintenance'
    if tour_purpose in ['Social/Rec', 'Eat Out', 'Discretionary', 'Home', 'Change Mode']:
        return 'Ind-Discretionary'
    return 'Ind-Discretionary'

spa_out_trips_df['grouped_tour_purpose'] = spa_out_trips_df['tour_purpose'].apply(lambda x: group_tour_purposes(x))

# joint status of 3 is fully joint tour
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_purpose'] == 'Ind-Maintenance') & (spa_out_trips_df['JOINT_STATUS'] == 3),
                 'grouped_tour_purpose'] = 'Joint-Maintenance'
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_purpose'] == 'Ind-Discretionary') & (spa_out_trips_df['JOINT_STATUS'] == 3),
                 'grouped_tour_purpose'] = 'Joint-Discretionary'
spa_out_trips_df['grouped_tour_purpose'].value_counts(dropna=False)

Work                   34377
Ind-Maintenance        32555
Ind-Discretionary      19858
School                 10396
Work sub-tour           4410
Joint-Discretionary     2841
Joint-Maintenance       2630
University              2144
Name: grouped_tour_purpose, dtype: int64

In [21]:
spa_out_trips_df['TRIPMODE'].value_counts(dropna=False)

1     47270
2     19809
3     13078
4     12028
6      8605
13     3001
5      1763
8      1349
11      672
7       634
14      360
10      355
12      249
9        38
Name: TRIPMODE, dtype: int64

In [22]:
spa_mode_dict = {
    1: 'DRIVEALONE',
    2: 'SHARED2',
    3: 'SHARED3',
    4: 'WALK',
    5: 'BIKE',
    6: 'WALK-TRANSIT',
    7: 'KNR-TRANSIT',
    8: 'PNR-TRANSIT',
    9: 'TNC-TRANSIT',
    10: 'TAXI',
    11: 'TNC-REG',
    12: 'TNC-SHARED',
    13: 'SCHOOLBUS',
    14: 'OTHER'
}
# # change taxi to shared2
# spa_tourmode_dict = {
#     1: 'DRIVEALONE',
#     2: 'SHARED2',
#     3: 'SHARED3',
#     4: 'WALK',
#     5: 'BIKE',
#     6: 'WALK-TRANSIT',
#     7: 'PNR-TRANSIT',
#     8: 'KNR-TRANSIT',
#     9: 'SCHOOLBUS',
#     10: 'SHARED2',
#     11: 'OTHER',
# }


spa_out_trips_df['linked_trip_mode'] = spa_out_trips_df['TRIPMODE'].apply(lambda x: spa_mode_dict[x])
spa_out_trips_df['tour_mode'] = spa_out_trips_df['TOURMODE'].apply(lambda x: spa_mode_dict[x])

In [23]:
print(spa_out_trips_df['tour_mode'].value_counts(dropna=False))
print(spa_out_trips_df['linked_trip_mode'].value_counts(dropna=False))


DRIVEALONE      38131
SHARED2         22242
SHARED3         18487
WALK-TRANSIT    11344
WALK             7969
SCHOOLBUS        4058
PNR-TRANSIT      2177
BIKE             1902
KNR-TRANSIT      1295
TNC-REG           642
TAXI              400
TNC-SHARED        290
OTHER             198
TNC-TRANSIT        76
Name: tour_mode, dtype: int64
DRIVEALONE      47270
SHARED2         19809
SHARED3         13078
WALK            12028
WALK-TRANSIT     8605
SCHOOLBUS        3001
BIKE             1763
PNR-TRANSIT      1349
TNC-REG           672
KNR-TRANSIT       634
OTHER             360
TAXI              355
TNC-SHARED        249
TNC-TRANSIT        38
Name: linked_trip_mode, dtype: int64


In [24]:
spa_out_trips_df['grouped_tour_mode'] = spa_out_trips_df['tour_mode']
spa_out_trips_df['grouped_linked_trip_mode'] = spa_out_trips_df['linked_trip_mode']
# spa_out_trips_df = group_transit_modes(df=spa_out_trips_df, mode='linked_trip_mode')

In [25]:
print(spa_out_trips_df['grouped_linked_trip_mode'].value_counts(dropna=False))
print(spa_out_trips_df['grouped_tour_mode'].value_counts(dropna=False))

DRIVEALONE      47270
SHARED2         19809
SHARED3         13078
WALK            12028
WALK-TRANSIT     8605
SCHOOLBUS        3001
BIKE             1763
PNR-TRANSIT      1349
TNC-REG           672
KNR-TRANSIT       634
OTHER             360
TAXI              355
TNC-SHARED        249
TNC-TRANSIT        38
Name: grouped_linked_trip_mode, dtype: int64
DRIVEALONE      38131
SHARED2         22242
SHARED3         18487
WALK-TRANSIT    11344
WALK             7969
SCHOOLBUS        4058
PNR-TRANSIT      2177
BIKE             1902
KNR-TRANSIT      1295
TNC-REG           642
TAXI              400
TNC-SHARED        290
OTHER             198
TNC-TRANSIT        76
Name: grouped_tour_mode, dtype: int64


In [26]:
spa_out_tours_df['tour_mode'] = spa_out_tours_df['TOURMODE'].apply(lambda x: spa_mode_dict[x])

In [27]:
spa_out_tours_df['spa_tour_purpose'] = spa_out_tours_df['TOURPURP'].apply(lambda x: spa_purp_dict[x])
spa_out_tours_df['tour_purpose'] = spa_out_tours_df['spa_tour_purpose'].apply(lambda x: spa_to_abms_purp_dict[x])
spa_out_tours_df.loc[spa_out_tours_df['IS_SUBTOUR'] == 1, 'tour_purpose'] = 'Work sub-tour'
spa_out_tours_df['grouped_tour_mode'] = spa_out_tours_df['tour_mode']

spa_out_tours_df['grouped_tour_purpose'] = spa_out_tours_df['tour_purpose'].apply(lambda x: group_tour_purposes(x))
# joint status of 3 is fully joint tour
spa_out_tours_df.loc[(spa_out_tours_df['grouped_tour_purpose'] == 'Ind-Maintenance')
                      & (spa_out_tours_df['JOINT_STATUS'] == 3),
                     'grouped_tour_purpose'] = 'Joint-Maintenance'
spa_out_tours_df.loc[(spa_out_tours_df['grouped_tour_purpose'] == 'Ind-Discretionary')
                      & (spa_out_tours_df['JOINT_STATUS'] == 3),
                     'grouped_tour_purpose'] = 'Joint-Discretionary'

In [28]:
spa_out_tours_df['tour_mode'].value_counts()

DRIVEALONE      14922
SHARED2          7857
SHARED3          6475
WALK-TRANSIT     4097
WALK             3925
SCHOOLBUS        1754
PNR-TRANSIT       809
BIKE              739
KNR-TRANSIT       447
TNC-REG           230
TAXI              145
TNC-SHARED        100
OTHER              87
TNC-TRANSIT        27
Name: tour_mode, dtype: int64

In [29]:
spa_out_tours_df['auto_suff'].value_counts(dropna=False)

2    33533
1     4896
0     3185
Name: auto_suff, dtype: int64

#### On-Board Transit Survey

In [30]:
obs_df['grouped_tour_mode'] = pd.NA
obs_df.loc[obs_df['TOUR_MODE'] == 'Walk', 'grouped_tour_mode'] = 'WALK-TRANSIT'
obs_df.loc[obs_df['TOUR_MODE'] == 'PNR', 'grouped_tour_mode'] = 'PNR-TRANSIT'
obs_df.loc[obs_df['TOUR_MODE'] == 'KNR', 'grouped_tour_mode'] = 'KNR-TRANSIT'
obs_df.loc[obs_df['TOUR_MODE'] == 'TNR', 'grouped_tour_mode'] = 'TNC-TRANSIT'

In [31]:
obs_df['grouped_tour_mode'].value_counts()


WALK-TRANSIT    90029
PNR-TRANSIT     68976
KNR-TRANSIT     21253
TNC-TRANSIT      2639
Name: grouped_tour_mode, dtype: int64

In [32]:
obs_df['TOUR_PURPOSE'].value_counts(dropna=False)
# obs_df[obs_df['TOUR_PURPOSE'].isnull()].head()
# There are some missing purposes, but they aren't weighted/can ignore

Work                   142946
Ind-Maintenance         13979
Ind-Discretionary        8804
University               8265
Work sub-tour            4210
School                   2586
Joint-Discretionary      1111
Joint-Maintenance         996
Name: TOUR_PURPOSE, dtype: int64

In [33]:
# Tour purpose in recoded OBS dataset already matches desired grouped tour purpose

obs_df['grouped_tour_purpose'] = obs_df['TOUR_PURPOSE']


### Cleaning Modes and Purposes

In [34]:
# All schoolbus trips should be on school tours
spa_out_tours_df.loc[spa_out_tours_df['grouped_tour_mode'] == 'SCHOOLBUS', 'grouped_tour_purpose'] = 'School'
spa_out_trips_df.loc[spa_out_trips_df['grouped_linked_trip_mode'] == 'SCHOOLBUS', 'trip_purpose'] = 'School'

spa_out_trips_df['tour_includes_schoolbus'] = spa_out_trips_df.groupby(
    ['HH_ID', 'PER_ID', 'TOUR_ID'])['grouped_linked_trip_mode'].transform(lambda x: 1 if 'SCHOOLBUS' in x.values else 0)

spa_out_trips_df.loc[spa_out_trips_df['tour_includes_schoolbus'] == 1, 'grouped_tour_purpose'] = 'School'

In [35]:
# Reallocate to appropriate cells

spa_out_trips_df['grouped_tour_mode_updated'] = np.nan



# Drive trips on bike tours should be drive tours
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_mode'] == 'BIKE') & \
                     (spa_out_trips_df['grouped_linked_trip_mode'].isin(['DRIVEALONE', 'SHARED2', 'SHARED3'])),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 
# Bike trips only on bike tour
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'BIKE'), 'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# Drive alone should be on drive tours only
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'].isin(['DRIVEALONE'])) &
                      (spa_out_trips_df['grouped_tour_mode'].isin(['TNC-REG', 'TNC-SHARED', 'TAXI', 'SCHOOLBUS'])),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# Drive on transit should be moved to drive  tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT','KNR-TRANSIT', 'TNC-TRANSIT']) & \
                     (spa_out_trips_df['grouped_linked_trip_mode'].isin(['DRIVEALONE', 'SHARED2', 'SHARED3']))),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# Transit on school bus should be transit 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT','KNR-TRANSIT', 'TNC-TRANSIT']) & \
                     (spa_out_trips_df['grouped_tour_mode'].isin(['SCHOOLBUS']))),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# KNR on TNR should be on KNR tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT')  & \
                    (spa_out_trips_df['grouped_tour_mode'] == 'PNR-TRANSIT') , 'grouped_tour_mode_updated'] = 'KNR-TRANSIT'

                     # Walk on TNR/ KNR/PNR should be on walk transit tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK')  & \
                       ((spa_out_trips_df['grouped_tour_mode'] == 'KNR-TRANSIT') | \
                        (spa_out_trips_df['grouped_tour_mode'] == 'PNR-TRANSIT') | \
                       (spa_out_trips_df['grouped_tour_mode'] == 'TNC-TRANSIT')) , 'grouped_tour_mode_updated'] = 'WALK-TRANSIT'


# TNR on KNR or PNR should be on TNR tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'TNC-TRANSIT')  & \
                       ((spa_out_trips_df['grouped_tour_mode'] == 'KNR-TRANSIT') | \
                        (spa_out_trips_df['grouped_tour_mode'] == 'PNR-TRANSIT')) , 'grouped_tour_mode_updated'] = 'TNC-TRANSIT'

# Walk on TNR/ KNR/PNR should be on walk transit tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT')  & \
                       ((spa_out_trips_df['grouped_tour_mode'] == 'KNR-TRANSIT') | \
                        (spa_out_trips_df['grouped_tour_mode'] == 'PNR-TRANSIT') | \
                       (spa_out_trips_df['grouped_tour_mode'] == 'TNC-TRANSIT')) , 'grouped_tour_mode_updated'] = 'WALK-TRANSIT'
# Walk trips on bike tour should be walk tour
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK')  & \
                    (spa_out_trips_df['grouped_tour_mode'] == 'BIKE') , 'grouped_tour_mode_updated'] = 'WALK'

       

In [36]:
# Recode tour mode

spa_out_trips_df.loc[(pd.notnull(spa_out_trips_df['grouped_tour_mode_updated'])), 'grouped_tour_mode'] = spa_out_trips_df['grouped_tour_mode_updated'] 

# Since the new tour modes do not really change the overall tour calibration targets, leaving those as-is.
# This means the data aren't 100% consistent between trips/tours when it comes to tour mode
# Updating the tour mode in the tours table (and then carrying that over to other trips on the tour) might cause more problems with working out which trip modes should be on which tour modes
#    so, best to avoid since it doesn't change the tour targets significantly.

In [37]:
display(spa_out_trips_df[(pd.notnull(spa_out_trips_df['grouped_tour_mode_updated']))])

,HH_ID,PER_ID,TOUR_ID,TRIP_ID,ORIG_PLACENO,ORIG_X,ORIG_Y,ORIG_TAZ,ORIG_MAZ,DEST_PLACENO,DEST_X,DEST_Y,DEST_TAZ,DEST_MAZ,ORIG_PURP,DEST_PURP,ORIG_ARR_HR,ORIG_ARR_MIN,ORIG_ARR_BIN,ORIG_DEP_HR,ORIG_DEP_MIN,ORIG_DEP_BIN,DEST_ARR_HR,DEST_ARR_MIN,DEST_ARR_BIN,DEST_DEP_HR,DEST_DEP_MIN,DEST_DEP_BIN,TRIP_DUR_HR,TRIP_DUR_MIN,TRIP_DUR_BIN,TRIPMODE,ISDRIVER,CHAUFFUER_ID,AUTO_OCC,TOURMODE,TOURPURP,BOARDING_PLACENO,BOARDING_PNAME,BOARDING_X,BOARDING_Y,BOARDING_TAP,ALIGHTING_PLACENO,ALIGHTING_PNAME,ALIGHTING_X,ALIGHTING_Y,ALIGHTING_TAP,TRANSIT_NUM_XFERS,TRANSIT_ROUTE_1,TRANSIT_MODE_1,XFER_1_PLACENO,XFER_1_PNAME,XFER_1_X,XFER_1_Y,XFER_1_TAP,TRANSIT_ROUTE_2,TRANSIT_MODE_2,XFER_2_PLACENO,XFER_2_PNAME,XFER_2_X,XFER_2_Y,XFER_2_TAP,TRANSIT_ROUTE_3,TRANSIT_MODE_3,XFER_3_PLACENO,XFER_3_PNAME,XFER_3_X,XFER_3_Y,XFER_3_TAP,PARKING_PLACENO,PARKING_PNAME,PARKING_X,PARKING_Y,SUBTOUR,IS_INBOUND,TRIPS_ON_JOURNEY,TRIPS_ON_TOUR,ORIG_IS_TOUR_ORIG,ORIG_IS_TOUR_DEST,DEST_IS_TOUR_DEST,DEST_IS_TOUR_ORIG,PEREXPFACT,HHEXPFACT,PERSONTYPE,FULLY_JOINT,PARTIAL_TOUR,JTRIP_ID,ESCORTED,ESCORTING,NUM_PERSONS_ESCORTED,ESCORT_PERS_1,ESCORT_PERS_2,ESCORT_PERS_3,ESCORT_PERS_4,ESCORT_PERS_5,DEST_ESCORTING,JOINT,NUM_UL_JTRIPS,DIST,ERROR,sampno_x,SUBREGCluster_balanced_weight,PER_WEIGHT,SAMPN,auto_suff,sampno_y,perno,PLANO,person_weight,trip_rate_factor,trip_weight,HH_WEIGHT_x,JOINT_STATUS,IS_SUBTOUR,HH_WEIGHT_y,unique_joint_trip,spa_tour_purpose,tour_purpose,grouped_tour_purpose,linked_trip_mode,tour_mode,grouped_tour_mode,grouped_linked_trip_mode,trip_purpose,tour_includes_schoolbus,grouped_tour_mode_updated
68,20001025,1,1,1,1,-87.696995,41.908001,173,NaN,2,-87.617090,41.894815,7,NaN,0,1,3,0,1,7,23,9,8,16,11,17,10,29,0,53,3,5,0,NaN,1,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,2,1,0,1,0,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,20001025,70.401912,70.401912,20001025,1,20001025,1,2,70.401912,1.000000,70.401912,70.401912,1,0,70.401912,NaN,WORK,Work,Work,BIKE,BIKE,BIKE,BIKE,NaN,0,BIKE
69,20001025,1,1,2,2,-87.617090,41.894815,7,NaN,3,-87.696995,41.908001,173,NaN,1,0,8,16,11,17,10,29,17,46,30,18,19,31,0,36,2,5,0,NaN,1,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,2,0,1,0,1,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,20001025,70.401912,70.401912,20001025,1,20001025,1,3,70.401912,1.000000,70.401912,70.401912,1,0,70.401912,NaN,WORK,Work,Work,BIKE,BIKE,BIKE,BIKE,NaN,0,BIKE
74,20001516,1,1,1,1,-87.905841,42.038083,483,NaN,2,-87.889312,42.042318,484,NaN,0,6,3,0,1,6,55,8,7,2,9,7,4,9,0,7,2,6,0,NaN,1,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,3,6,1,0,0,0,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,20001516,101.706666,123.960858,20001516,2,20001516,1,2,101.706666,1.218808,123.960858,123.960858,1,0,152.946558,NaN,WORK,Work,Work,WALK-TRANSIT,PNR-TRANSIT,WALK-TRANSIT,WALK-TRANSIT,NaN,0,WALK-TRANSIT
75,20001516,1,1,2,2,-87.889312,42.042318,484,NaN,3,-87.886607,42.040965,484,NaN,6,6,7,2,9,7,4,9,7,7,9,7,11,9,0,3,1,6,0,NaN,1,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,3,6,0,0,0,0,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,20001516,101.706666,168.020985,20001516,2,20001516,1,3,101.706666,1.652015,168.020985,168.020985,1,0,152.946558,NaN,WORK,Work,Work,WALK-TRANSIT,PNR-TRANSIT,WALK-TRANSIT,WALK-TRANSIT,NaN,0,WALK-TRANSIT
77,20001516,1,1,4,4,-87.637440,41.881186,28,NaN,5,-87.640533,41.883027,21,NaN,1,6,7,53,10,16,22,27,16,28,27,16,39,28,0,6,1,6,0,NaN,1,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [38]:
def remove_other_from_mode(df, mode, data_source):
    print("removing ", len(df[df[mode] == 'OTHER']), data_source, mode, "entries with OTHER mode")
    return df[df[mode] != 'OTHER']

# Remove OTHER trip and tour modes since they are not included in the model
spa_out_tours_df = remove_other_from_mode(spa_out_tours_df, mode='grouped_tour_mode', data_source='HTS')
spa_out_trips_df = remove_other_from_mode(spa_out_trips_df, mode='grouped_linked_trip_mode', data_source='HTS')
spa_out_trips_df = remove_other_from_mode(spa_out_trips_df, mode='grouped_tour_mode', data_source='HTS')


removing  87 HTS grouped_tour_mode entries with OTHER mode
removing  360 HTS grouped_linked_trip_mode entries with OTHER mode
removing  0 HTS grouped_tour_mode entries with OTHER mode


In [39]:
spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'SCHOOLBUS']['grouped_tour_purpose'].value_counts()


School    3001
Name: grouped_tour_purpose, dtype: int64

## Trip Mode Choice

In [40]:
spa_out_tours_df.grouped_tour_mode.value_counts()

DRIVEALONE      14922
SHARED2          7857
SHARED3          6475
WALK-TRANSIT     4097
WALK             3925
SCHOOLBUS        1754
PNR-TRANSIT       809
BIKE              739
KNR-TRANSIT       447
TNC-REG           230
TAXI              145
TNC-SHARED        100
TNC-TRANSIT        27
Name: grouped_tour_mode, dtype: int64

In [41]:
spa_out_tour_trips_ct = pd.crosstab(
    spa_out_trips_df['grouped_linked_trip_mode'],
    spa_out_trips_df['grouped_tour_mode'],
    values=spa_out_trips_df['PER_WEIGHT'],
    aggfunc=sum,
    margins=True,
)

spa_out_tour_trips_ct = spa_out_tour_trips_ct.reindex(
    index=output_calibration_modes, columns=output_calibration_modes, fill_value=0)
round(spa_out_tour_trips_ct).fillna('-')

grouped_tour_mode,DRIVEALONE,SHARED2,SHARED3,WALK,BIKE,WALK-TRANSIT,PNR-TRANSIT,KNR-TRANSIT,TNC-TRANSIT,TAXI,TNC-REG,TNC-SHARED,SCHOOLBUS,All
grouped_linked_trip_mode,,,,,,,,,,,,,,
DRIVEALONE,13415263.0,2265664.0,906413.0,-,-,-,-,-,-,-,-,-,-,16587341.0
SHARED2,-,5892131.0,1062605.0,-,-,-,-,-,-,23433.0,14162.0,7648.0,105825.0,7105804.0
SHARED3,-,-,4088340.0,-,-,-,-,-,-,7866.0,9395.0,1976.0,101340.0,4208916.0
WALK,171395.0,165892.0,140496.0,2633249.0,-,507369.0,-,-,-,6646.0,52909.0,18249.0,32468.0,3728673.0
BIKE,-,-,-,-,377237.0,-,-,-,-,-,-,-,-,377237.0
WALK-TRANSIT,-,-,-,-,-,1768583.0,-,-,-,-,-,-,-,1768583.0
PNR-TRANSIT,-,-,-,-,-,-,221014.0,-,-,-,-,-,-,221014.0
KNR-TRANSIT,-,-,-,-,-,-,-,145621.0,-,-,-,-,-,145621.0
TNC-TRANSIT,-,-,-,-,-,-,-,-,7224.0,-,-,-,-,7224.0


Activity based modeling data is already in this format:

#### Writing these to trip mode choice excel sheet

In [42]:
def write_tables_to_excel(dfs, excel_writer, excel_sheet_name, start_row, start_col, title, sep_for_col_title, col_title):
    # have to write first table to initialize sheet before writing title
    dfs[0].to_excel(excel_writer, excel_sheet_name, startrow=start_row+2, startcol=start_col)
    worksheet = excel_writer.sheets[excel_sheet_name]
    
    # writing title at and first table name
    worksheet.write(start_row, start_col, title)
    worksheet.write(start_row+1, start_col, dfs[0].name)
    worksheet.write(start_row+1, start_col + sep_for_col_title, col_title)
    start_row += len(dfs[0]) + 6
    
    for df in dfs[1:]:
        df.to_excel(excel_writer, excel_sheet_name, startrow=start_row, startcol=start_col)
        worksheet.write(start_row-1, start_col, df.name)
        worksheet.write(start_row-1, start_col + sep_for_col_title, col_title)
        start_row += len(df) + 4

In [43]:
def create_hts_trip_mode_choice_tables(spa_out_trips_df, purposes):
    dfs = []
    all_purposes_ct = pd.crosstab(
        spa_out_trips_df['grouped_linked_trip_mode'],
        spa_out_trips_df['grouped_tour_mode'],
        values=spa_out_trips_df['PER_WEIGHT'],
        aggfunc='sum',
        margins=True,
        dropna=False
    )
    all_purposes_ct = all_purposes_ct.reindex(
            index=output_calibration_modes, columns=output_calibration_modes, fill_value=0)
    all_purposes_ct = round(all_purposes_ct.fillna(0),0)
    
    all_purposes_ct.name = 'Purpose: Total'
    dfs.append(all_purposes_ct)
    
    # Create crosstab for each purpose
    for tour_purpose in purposes:
        trip_tour_mode_by_purp_ct = pd.crosstab(
            spa_out_trips_df[(spa_out_trips_df['grouped_tour_purpose'] == tour_purpose)]['grouped_linked_trip_mode'],
            spa_out_trips_df[(spa_out_trips_df['grouped_tour_purpose'] == tour_purpose)]['grouped_tour_mode'],
            values=spa_out_trips_df[(spa_out_trips_df['grouped_tour_purpose'] == tour_purpose)]['PER_WEIGHT'],
            aggfunc='sum',
            margins=True,
            dropna=False
        )
        trip_tour_mode_by_purp_ct = trip_tour_mode_by_purp_ct.reindex(
            index=output_calibration_modes, columns=output_calibration_modes, fill_value=0)
        trip_tour_mode_by_purp_ct = round(trip_tour_mode_by_purp_ct.fillna(0),0)
        
        trip_tour_mode_by_purp_ct.name = 'Purpose: ' + tour_purpose
        dfs.append(trip_tour_mode_by_purp_ct)
        
    return dfs


In [44]:
spa_out_trips_df['grouped_tour_purpose'].value_counts()


Work                   34261
Ind-Maintenance        32204
Ind-Discretionary      19559
School                 10880
Work sub-tour           4379
Joint-Discretionary     2839
Joint-Maintenance       2628
University              2101
Name: grouped_tour_purpose, dtype: int64

In [45]:
print(output_calibration_purposes[:-1])
spa_out_trips_df['grouped_tour_mode'].value_counts()


['Work', 'University', 'School', 'Work sub-tour', 'Ind-Maintenance', 'Ind-Discretionary', 'Joint-Maintenance', 'Joint-Discretionary']


DRIVEALONE      38822
SHARED2         22777
SHARED3         18797
WALK-TRANSIT    11338
WALK             8100
SCHOOLBUS        3952
BIKE             1763
PNR-TRANSIT      1363
KNR-TRANSIT       666
TNC-REG           599
TAXI              352
TNC-SHARED        275
TNC-TRANSIT        47
Name: grouped_tour_mode, dtype: int64

In [46]:
hts_trip_mc_dfs = create_hts_trip_mode_choice_tables(spa_out_trips_df, output_calibration_purposes[:-1])

## Tour Mode Choice

In [47]:
walk_tours = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'] == 'WALK-TRANSIT']['PER_WEIGHT'].sum()
walk_trips = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT']['PER_WEIGHT'].sum()
walk_trips_per_tour = walk_trips / walk_tours

walk_transit_trips = spa_out_trips_df[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT') 
                                  & (spa_out_trips_df['TRIPMODE'].isin(range(6, 9 + 1)))]['PER_WEIGHT'].sum()

walk_transit_trips_per_tour = walk_transit_trips / walk_tours

pnr_tours = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'] == 'PNR-TRANSIT']['PER_WEIGHT'].sum()
pnr_trips = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT']['PER_WEIGHT'].sum()
pnr_trips_per_tour = pnr_trips / pnr_tours
pnr_transit_trips = spa_out_trips_df[(spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT') 
                                  & (spa_out_trips_df['TRIPMODE'].isin(range(6, 9 + 1)))]['PER_WEIGHT'].sum()

pnr_transit_trips_per_tour = pnr_transit_trips / pnr_tours

knr_tours = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'] == 'KNR-TRANSIT']['PER_WEIGHT'].sum()
knr_trips = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT']['PER_WEIGHT'].sum()
knr_trips_per_tour = knr_trips / knr_tours

knr_transit_trips = spa_out_trips_df[(spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT') 
                                  & (spa_out_trips_df['TRIPMODE'].isin(range(6, 9 + 1)))]['PER_WEIGHT'].sum()

knr_transit_trips_per_tour = knr_transit_trips / knr_tours

tnr_tours = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'] == 'TNC-TRANSIT']['PER_WEIGHT'].sum()
tnr_trips = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'TNC-TRANSIT']['PER_WEIGHT'].sum()
tnr_trips_per_tour = tnr_trips / tnr_tours

tnr_transit_trips = spa_out_trips_df[(spa_out_trips_df['grouped_linked_trip_mode'] == 'TNC-TRANSIT') 
                                  & (spa_out_trips_df['TRIPMODE'].isin(range(6, 9 + 1)))]['PER_WEIGHT'].sum()

tnr_transit_trips_per_tour = tnr_transit_trips / tnr_tours

print("Trips Per Tour for ")
print("WALK-TRANSIT Tours:", walk_tours, "\t Linked Trips:", walk_trips,
      "\t Trips/Tour:",  round(walk_trips_per_tour, 3), "\t Transit Trips/Tour: ", round(walk_transit_trips_per_tour,3))
print("PNR-TRANSIT Tours:", pnr_tours, "\t Linked Trips:", pnr_trips,
      "\t Trips/Tour:",  round(pnr_trips_per_tour, 3), "\t Transit Trips/Tour: ", round(pnr_transit_trips_per_tour,3))
print("KNR-TRANSIT Tours:", knr_tours, "\t Linked Trips:", knr_trips,
      "\t Trips/Tour:",  round(knr_trips_per_tour, 3), "\t Transit Trips/Tour: ", round(knr_transit_trips_per_tour,3))
print("TNC-TRANSIT Tours:", tnr_tours, "\t Linked Trips:", tnr_trips,
      "\t Trips/Tour:",  round(tnr_trips_per_tour, 3), "\t Transit Trips/Tour: ", round(tnr_transit_trips_per_tour,3))

# tnr and knr are the same; but it looks like that's correct

Trips Per Tour for 
WALK-TRANSIT Tours: 832862.6998510286 	 Linked Trips: 1768582.7359054282 	 Trips/Tour: 2.123 	 Transit Trips/Tour:  2.123
PNR-TRANSIT Tours: 131353.2525621925 	 Linked Trips: 221014.19606790994 	 Trips/Tour: 1.683 	 Transit Trips/Tour:  1.683
KNR-TRANSIT Tours: 101252.47957393755 	 Linked Trips: 145621.48689890743 	 Trips/Tour: 1.438 	 Transit Trips/Tour:  1.438
TNC-TRANSIT Tours: 5022.499855874316 	 Linked Trips: 7223.819610647261 	 Trips/Tour: 1.438 	 Transit Trips/Tour:  1.438


In [48]:
def get_transit_trips_per_tour(df, grouped_tour_mode):
    tours = df[df['grouped_tour_mode'] == grouped_tour_mode]['PER_WEIGHT'].sum()
    transit_trips = df[(df['grouped_tour_mode'] == goruped_tour_mode)
                      & (df['TRIPMODE'].isin(range(9, 17 + 1)))]['PER_WEIGHT'].sum()
    return transit_trips/tours

transit_trips_per_transit_tour_df = pd.DataFrame(index=output_calibration_modes)
transit_trips_per_transit_tour_df['transit_trips_per_transit_tour'] = pd.NA
transit_trips_per_transit_tour_df.loc['WALK-TRANSIT', 'transit_trips_per_transit_tour'] = walk_transit_trips_per_tour
transit_trips_per_transit_tour_df.loc['PNR-TRANSIT', 'transit_trips_per_transit_tour'] = pnr_transit_trips_per_tour
transit_trips_per_transit_tour_df.loc['KNR-TRANSIT', 'transit_trips_per_transit_tour'] = knr_transit_trips_per_tour
transit_trips_per_transit_tour_df.loc['TNC-TRANSIT', 'transit_trips_per_transit_tour'] = tnr_transit_trips_per_tour
transit_trips_per_transit_tour_df.name = 'Purpose: Total'
transit_trips_per_transit_tour_df

# Not high enough -- replace pnr/knr with 2.0
transit_trips_per_transit_tour_df.loc['PNR-TRANSIT', 'transit_trips_per_transit_tour'] = 2.0
transit_trips_per_transit_tour_df.loc['KNR-TRANSIT', 'transit_trips_per_transit_tour'] = 2.0
transit_trips_per_transit_tour_df.loc['TNC-TRANSIT', 'transit_trips_per_transit_tour'] = 2.0


In [49]:
transit_trips_per_transit_tour_df

,transit_trips_per_transit_tour
DRIVEALONE,<NA>
SHARED2,<NA>
SHARED3,<NA>
WALK,<NA>
BIKE,<NA>
WALK-TRANSIT,2.123499
PNR-TRANSIT,2.0
KNR-TRANSIT,2.0
TNC-TRANSIT,2.0
TAXI,<NA>


In [50]:
obs_df['tourweight'] = obs_df['linked_weight_scaled']
obs_df.loc[obs_df['grouped_tour_mode'] == 'WALK-TRANSIT','tourweight'] =  obs_df.loc[
    obs_df['grouped_tour_mode'] == 'WALK-TRANSIT', 'tourweight'] / transit_trips_per_transit_tour_df.loc[
    'WALK-TRANSIT', 'transit_trips_per_transit_tour']
obs_df.loc[obs_df['grouped_tour_mode'] == 'PNR-TRANSIT', 'tourweight'] =  obs_df.loc[
    obs_df['grouped_tour_mode'] == 'PNR-TRANSIT', 'tourweight'] / transit_trips_per_transit_tour_df.loc[
    'PNR-TRANSIT', 'transit_trips_per_transit_tour']
obs_df.loc[obs_df['grouped_tour_mode'] == 'KNR-TRANSIT', 'tourweight'] =  obs_df.loc[
    obs_df['grouped_tour_mode'] == 'KNR-TRANSIT', 'tourweight'] / transit_trips_per_transit_tour_df.loc[
    'KNR-TRANSIT', 'transit_trips_per_transit_tour']
obs_df.loc[obs_df['grouped_tour_mode'] == 'TNC-TRANSIT', 'tourweight'] =  obs_df.loc[
    obs_df['grouped_tour_mode'] == 'TNC-TRANSIT', 'tourweight'] / transit_trips_per_transit_tour_df.loc[
    'TNC-TRANSIT', 'transit_trips_per_transit_tour']
print("Total number of On-board survey Tours: ", int(obs_df['tourweight'].sum()))
print("Total number of On-board survey Trips: ", int(obs_df['linked_weight_scaled'].sum()))

Total number of On-board survey Tours:  597731
Total number of On-board survey Trips:  1251046


In [51]:
obs_df.grouped_tour_mode.value_counts()

WALK-TRANSIT    90029
PNR-TRANSIT     68976
KNR-TRANSIT     21253
TNC-TRANSIT      2639
Name: grouped_tour_mode, dtype: int64

In [52]:
def create_hts_tour_mode_choice_tables(spa_out_tours_df, purposes):
    dfs = []
    all_purposes_ct = pd.crosstab(
        spa_out_tours_df['grouped_tour_mode'],
        spa_out_tours_df['auto_suff'],
        values=spa_out_tours_df['PER_WEIGHT'],
        aggfunc='sum',
        margins=True,
        dropna=False
    )
    all_purposes_ct = all_purposes_ct.reindex(
            index=output_calibration_modes, fill_value=0)
    all_purposes_ct = round(all_purposes_ct.fillna(0),0)
    
    all_purposes_ct.name = 'Purpose: Total'
    dfs.append(all_purposes_ct)
    
    # Create crosstab for each purpose
    for tour_purpose in purposes:
        tour_mode_autosuff_by_purp_ct = pd.crosstab(
            spa_out_tours_df[(spa_out_tours_df['grouped_tour_purpose'] == tour_purpose)]['grouped_tour_mode'],
            spa_out_tours_df[(spa_out_tours_df['grouped_tour_purpose'] == tour_purpose)]['auto_suff'],
            values=spa_out_tours_df[(spa_out_tours_df['grouped_tour_purpose'] == tour_purpose)]['PER_WEIGHT'],
            aggfunc='sum',
            margins=True,
            dropna=False
        )
        tour_mode_autosuff_by_purp_ct = tour_mode_autosuff_by_purp_ct.reindex(
            index=output_calibration_modes, fill_value=0)
        tour_mode_autosuff_by_purp_ct = round(tour_mode_autosuff_by_purp_ct.fillna(0),0)
        
        tour_mode_autosuff_by_purp_ct.name = 'Purpose: ' + tour_purpose
        dfs.append(tour_mode_autosuff_by_purp_ct)
        
    return dfs


def create_obs_tour_mode_choice_tables(obs_df, purposes, weight):
    dfs = []
    all_purposes_ct = pd.crosstab(
        obs_df['grouped_tour_mode'],
        obs_df['auto_suff'],
        values=obs_df[weight],
        aggfunc='sum',
        margins=True,
        dropna=False
    )
    all_purposes_ct = all_purposes_ct.reindex(
            index=output_calibration_modes, fill_value=0)
    all_purposes_ct = round(all_purposes_ct.fillna(0),0)
    
    all_purposes_ct.name = 'Purpose: Total'
    dfs.append(all_purposes_ct)
    
    # Create crosstab for each purpose
    for tour_purpose in purposes:
        tour_mode_autosuff_by_purp_ct = pd.crosstab(
            obs_df[(obs_df['grouped_tour_purpose'] == tour_purpose)]['grouped_tour_mode'],
            obs_df[(obs_df['grouped_tour_purpose'] == tour_purpose)]['auto_suff'],
            values=obs_df[weight],
            aggfunc='sum',
            margins=True,
            dropna=False
        )
        tour_mode_autosuff_by_purp_ct = tour_mode_autosuff_by_purp_ct.reindex(
            index=output_calibration_modes, fill_value=0)
        tour_mode_autosuff_by_purp_ct = round(tour_mode_autosuff_by_purp_ct.fillna(0),0)
        
        tour_mode_autosuff_by_purp_ct.name = 'Purpose: ' + tour_purpose
        dfs.append(tour_mode_autosuff_by_purp_ct)
        
    return dfs

In [53]:
def split_obs_ind_to_joint_plus_ind(obs_tour_mc_dfs, hts_transit_df):
    joint_disc_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Joint-Discretionary']['PER_WEIGHT'].sum()
    ind_disc_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Ind-Discretionary']['PER_WEIGHT'].sum()
    percent_ind_disc_tours = ind_disc_tours / (joint_disc_tours + ind_disc_tours)
    
    joint_maint_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Joint-Maintenance']['PER_WEIGHT'].sum()
    ind_maint_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Ind-Maintenance']['PER_WEIGHT'].sum()
    percent_ind_maint_tours = ind_maint_tours / (joint_maint_tours + ind_maint_tours)
    
    print("Household Travel Survey has")
    print("Maintenance: ", round(percent_ind_disc_tours*100,2), '% Individual, ', round((1-percent_ind_disc_tours)*100,2), '% Joint')
    print("Discretionary: ", round(percent_ind_maint_tours*100,2), '% Individual, ', round((1-percent_ind_maint_tours)*100,2), '% Joint')

    output_dfs = []
    ind_maint_df = pd.DataFrame()
    ind_disc_df = pd.DataFrame()
    joint_maint_df = pd.DataFrame()
    joint_disc_df = pd.DataFrame()
    
    for df in obs_tour_mc_dfs:
        if 'Ind-Maintenance' in df.name:
            ind_maint_df = round(df * percent_ind_maint_tours)
            ind_maint_df.name = 'Purpose: Ind-Maintenance'
            joint_maint_df = round(df * (1 - percent_ind_maint_tours))
            joint_maint_df.name = 'Purpose: Joint-Maintenance'
        elif 'Ind-Discretionary' in df.name:
            ind_disc_df = round(df * percent_ind_disc_tours)
            ind_disc_df.name = 'Purpose: Ind-Discretionary'
            joint_disc_df = round(df * (1 - percent_ind_disc_tours))
            joint_disc_df.name = 'Purpose: Joint-Discretionary'
        else:
            output_dfs.append(df)
            
    output_dfs.append(ind_maint_df)
    output_dfs.append(ind_disc_df)
    output_dfs.append(joint_maint_df)
    output_dfs.append(joint_disc_df)
    
    total_tours_before_split = sum([df.sum().sum() for df in obs_tour_mc_dfs])
    total_tours_after_split = sum([df.sum().sum() for df in output_dfs])
    assert abs(total_tours_before_split - total_tours_after_split) < 1, "Tours not conserved!!!"
            
    return output_dfs

In [54]:
hts_tour_mc_dfs = create_hts_tour_mode_choice_tables(spa_out_tours_df, output_calibration_purposes[:-1])
obs_tour_mc_trips_dfs = create_obs_tour_mode_choice_tables(obs_df, output_calibration_purposes[:-1], weight='linked_weight_scaled')
obs_tour_mc_tours_dfs = create_obs_tour_mode_choice_tables(obs_df, output_calibration_purposes[:-1], weight='tourweight')

# splitting obs maintenance and discretionary tour purposes into joint and ind based on hts split
# hts_transit_trips_df = spa_out_trips_df[spa_out_trips_df['grouped_tour_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT'])]
# obs_tour_mc_trips_dfs = split_obs_ind_to_joint_plus_ind(obs_tour_mc_trips_dfs, hts_transit_trips_df)

# hts_transit_tours_df = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT'])]
# obs_tour_mc_tours_dfs = split_obs_ind_to_joint_plus_ind(obs_tour_mc_tours_dfs, hts_transit_tours_df)

In [55]:
obs_tour_mc_tours_dfs

[auto_suff                 0         1         2       All
 grouped_tour_mode                                        
 DRIVEALONE              0.0       0.0       0.0       0.0
 SHARED2                 0.0       0.0       0.0       0.0
 SHARED3                 0.0       0.0       0.0       0.0
 WALK                    0.0       0.0       0.0       0.0
 BIKE                    0.0       0.0       0.0       0.0
 WALK-TRANSIT       157353.0  116933.0  175780.0  450066.0
 PNR-TRANSIT          1964.0    8540.0   84200.0   94705.0
 KNR-TRANSIT          5585.0   11020.0   28650.0   45255.0
 TNC-TRANSIT          2178.0    1554.0    3975.0    7707.0
 TAXI                    0.0       0.0       0.0       0.0
 TNC-REG                 0.0       0.0       0.0       0.0
 TNC-SHARED              0.0       0.0       0.0       0.0
 SCHOOLBUS               0.0       0.0       0.0       0.0
 All                167079.0  138047.0  292606.0  597732.0,
 auto_suff                0        1         2       Al

In [56]:
def create_final_tour_mc_targets(obs_tour_mc_tours_dfs, hts_tour_mc_dfs):
    tour_mc_calib_targets_dfs = []
    tour_mc_scaling_factors_dfs = []
    tour_mc_calib_targets_pct_dfs = []
    for i in range(len(obs_tour_mc_tours_dfs)):
        obs_tours_df = obs_tour_mc_tours_dfs[i]
        hts_tours_df = hts_tour_mc_dfs[i]
        assert obs_tours_df.name == hts_tours_df.name, "Purposes not the same!"
        
        # building calibration targets
        calibration_targets_df = hts_tours_df.copy()
        calibration_targets_df.name = hts_tours_df.name
        
        calibration_targets_df.loc[calibration_targets_df.index.isin(
            ['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT', 'TNC-TRANSIT'])] = obs_tours_df.loc[obs_tours_df.index.isin(
            ['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT','TNC-TRANSIT'])]
        calibration_targets_df.loc['All'] = calibration_targets_df.drop(labels='All', axis=0, inplace=False).sum()
        tour_mc_calib_targets_dfs.append(calibration_targets_df)
        
        # building scaling factors
        scaling_factors_df = pd.DataFrame(index=output_calibration_modes)
        scaling_factors_df['scaling_factor'] = pd.NA
        scaling_factors_df.loc['WALK-TRANSIT', 'scaling_factor'] = calibration_targets_df.loc['WALK-TRANSIT', 'All'] / hts_tours_df.loc['WALK-TRANSIT', 'All']
        scaling_factors_df.loc['PNR-TRANSIT', 'scaling_factor'] = calibration_targets_df.loc['PNR-TRANSIT', 'All'] / hts_tours_df.loc['PNR-TRANSIT', 'All']
        scaling_factors_df.loc['KNR-TRANSIT', 'scaling_factor'] = calibration_targets_df.loc['KNR-TRANSIT', 'All'] / hts_tours_df.loc['KNR-TRANSIT', 'All']
        scaling_factors_df.loc['TNC-TRANSIT', 'scaling_factor'] = calibration_targets_df.loc['TNC-TRANSIT', 'All'] / hts_tours_df.loc['TNC-TRANSIT', 'All']
        scaling_factors_df.replace(to_replace=np.inf, value=0, inplace=True)
        scaling_factors_df.name = calibration_targets_df.name
        tour_mc_scaling_factors_dfs.append(scaling_factors_df)
        
        # create distribution table
        calibration_targets_pct_df = calibration_targets_df.copy()
        calibration_targets_pct_df.drop(labels='All', axis=0, inplace=True)
        cols = list(calibration_targets_pct_df.columns)
        calibration_targets_pct_df[cols] = calibration_targets_pct_df[cols].div(calibration_targets_pct_df[cols].sum(axis=0), axis=1)

        #trip_mc_calib_targets_pct_df.loc['All'] = trip_mc_calib_targets_pct_df.sum()
        calibration_targets_pct_df.loc['All'] = calibration_targets_pct_df.sum(axis=0)
        
        calibration_targets_pct_df.name = calibration_targets_df.name
        
        tour_mc_calib_targets_pct_dfs.append(calibration_targets_pct_df)
        
    return tour_mc_calib_targets_dfs, tour_mc_scaling_factors_dfs, tour_mc_calib_targets_pct_dfs
        


In [57]:
tour_mc_calib_targets_dfs, tour_mc_scaling_factors_dfs, tour_mc_calib_targets_pct_dfs = create_final_tour_mc_targets(obs_tour_mc_tours_dfs, hts_tour_mc_dfs)

C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars


In [58]:
# Creating a table that is the sum of all non-Total purposes for consistency check
tour_mc_summed_purposes_df = tour_mc_calib_targets_dfs[1].copy()
for i in range(2, len(tour_mc_calib_targets_dfs)):
    tour_mc_summed_purposes_df = tour_mc_summed_purposes_df + tour_mc_calib_targets_dfs[i]
tour_mc_summed_purposes_df.name = 'Non-Total Sum'
tour_mc_calib_targets_dfs.append(tour_mc_summed_purposes_df)

In [59]:
tour_mc_scaling_factors_dfs

[             scaling_factor
 DRIVEALONE             <NA>
 SHARED2                <NA>
 SHARED3                <NA>
 WALK                   <NA>
 BIKE                   <NA>
 WALK-TRANSIT       0.540384
 PNR-TRANSIT        0.720996
 KNR-TRANSIT        0.446954
 TNC-TRANSIT        1.534648
 TAXI                   <NA>
 TNC-REG                <NA>
 TNC-SHARED             <NA>
 SCHOOLBUS              <NA>
 All                    <NA>,
              scaling_factor
 DRIVEALONE             <NA>
 SHARED2                <NA>
 SHARED3                <NA>
 WALK                   <NA>
 BIKE                   <NA>
 WALK-TRANSIT       0.852301
 PNR-TRANSIT        0.886834
 KNR-TRANSIT        0.741243
 TNC-TRANSIT        2.365617
 TAXI                   <NA>
 TNC-REG                <NA>
 TNC-SHARED             <NA>
 SCHOOLBUS              <NA>
 All                    <NA>,
              scaling_factor
 DRIVEALONE             <NA>
 SHARED2                <NA>
 SHARED3                <NA>
 WALK       

In [60]:
# Writing Tour Mode Choice targets
# HTS tours
write_tables_to_excel(
    dfs=hts_tour_mc_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=0,
    title='HTS Survey Tours',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)

# OBS trips
write_tables_to_excel(
    dfs=obs_tour_mc_trips_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=7,
    title='On-Board Survey Trips',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)

# Transit Trips per Transit tour
# using the same scaling factor for all trip purposes
transit_trips_per_transit_tour_dfs = [transit_trips_per_transit_tour_df for i in range(len(obs_tour_mc_trips_dfs))]
write_tables_to_excel(
    dfs=transit_trips_per_transit_tour_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=13,
    title='OBS Trips to Tours Factors',
    sep_for_col_title=1,
    col_title=''
)

# OBS tours
write_tables_to_excel(
    dfs=obs_tour_mc_tours_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=16,
    title='On-Board Survey Tours',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)

# Calibration Targets
write_tables_to_excel(
    dfs=tour_mc_calib_targets_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=23,
    title='Calibration Targets',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)


# Calibration Targets distributions
write_tables_to_excel(
    dfs=tour_mc_calib_targets_pct_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=30,
    title='Calibration Targets - Distribution',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)


### Creating final Trip Mode Choice targets

In [61]:
def create_trip_mode_choice_targets(hts_trip_mc_dfs, obs_tour_mc_trips_dfs):
    trip_mc_calib_targets_dfs = []
    obs_trip_target_dfs = []
    trip_mc_scaling_factors_dfs = []
    hts_trip_mc_dfs_updated = []
    trip_mc_calib_targets_pct_dfs = []
    for i in range(len(obs_tour_mc_tours_dfs)):
        hts_trip_tour_mode_df = hts_trip_mc_dfs[i]
        obs_trips_df = obs_tour_mc_trips_dfs[i]
        assert obs_trips_df.name == hts_trip_tour_mode_df.name, "Purposes not the same!"
        
        # Copying OBS trip targets and Scaling
        obs_trip_target_df = pd.DataFrame(index=output_calibration_modes)
        obs_trip_target_df['OBS_trips'] = pd.NA
        obs_trip_target_df['row_scaling_factor'] = pd.NA
        obs_trip_target_df['col_scaling_factor'] = pd.NA
        trip_mc_calib_targets_df = hts_trip_tour_mode_df.copy()

        transit_modes = ['WALK-TRANSIT', 'PNR-TRANSIT', 'KNR-TRANSIT', 'TNC-TRANSIT']
        
        # display(hts_trip_tour_mode_df)
        for transit_mode in transit_modes:

     
            # final trip targets come from OBS
            obs_trip_target_df.loc[transit_mode, 'OBS_trips'] = obs_trips_df.loc[transit_mode, 'All']
            # scaling linked-trips to match trip targets from OBS
            obs_trip_target_df.loc[transit_mode, 'row_scaling_factor'] = \
                obs_trip_target_df.loc[transit_mode, 'OBS_trips'] / hts_trip_tour_mode_df.loc[transit_mode, 'All']
            trip_mc_calib_targets_df.loc[transit_mode] = \
                trip_mc_calib_targets_df.loc[transit_mode] * obs_trip_target_df.loc[transit_mode, 'row_scaling_factor']

            # need to scale such that the trip distribution between non-transit and transit trips 
            #  for the transit mode remain the same

            pre_scaled_transit_tours = hts_trip_tour_mode_df.loc[transit_modes, transit_mode].sum()
            print(pre_scaled_transit_tours)
            scaled_transit_tours = trip_mc_calib_targets_df.loc[transit_modes, transit_mode].sum()
            print(scaled_transit_tours)

            # creating scaling factor for non-transit trips in transit tours
            obs_trip_target_df.loc[transit_mode, 'col_scaling_factor'] = scaled_transit_tours / pre_scaled_transit_tours
            print(obs_trip_target_df)
            # applying that scaling factor to non-transit trips
            trip_mc_calib_targets_df.loc[~hts_trip_tour_mode_df.index.isin(transit_modes), transit_mode] = \
                    hts_trip_tour_mode_df.loc[~hts_trip_tour_mode_df.index.isin(transit_modes), transit_mode] \
                    * obs_trip_target_df.loc[transit_mode, 'col_scaling_factor']
            
            # when there are no transit trips in HTS, but there are targets in OBS survey, all OBS trips are assumed symmetric
            if (hts_trip_tour_mode_df.loc[transit_mode, transit_modes].sum() == 0):
                trip_mc_calib_targets_df.loc[transit_mode, transit_mode] = obs_trip_target_df.loc[transit_mode, 'OBS_trips']
        

        
        obs_trip_target_df.loc['All', 'OBS_trips'] = obs_trip_target_df['OBS_trips'].sum()
        obs_trip_target_df.replace(to_replace=np.inf, value=0, inplace=True)
        obs_trip_target_df.name = obs_trips_df.name
        obs_trip_target_dfs.append(obs_trip_target_df)
        

        trip_mc_calib_targets_df.fillna(0, inplace=True)

        trip_mc_calib_targets_df.loc['All'] = trip_mc_calib_targets_df.drop(labels='All', axis=0, inplace=False).sum()
        trip_mc_calib_targets_df.loc[:,'All'] = trip_mc_calib_targets_df.drop(labels='All', axis=1, inplace=False).sum(axis=1)
        trip_mc_calib_targets_df = round(trip_mc_calib_targets_df)
        trip_mc_calib_targets_df.name = obs_trip_target_df.name
        
        # create distribution table
        trip_mc_calib_targets_pct_df = trip_mc_calib_targets_df.copy()
        trip_mc_calib_targets_pct_df.drop(labels='All', axis=0, inplace=True)
        cols = list(trip_mc_calib_targets_pct_df.columns)
        trip_mc_calib_targets_pct_df[cols] = trip_mc_calib_targets_pct_df[cols].div(trip_mc_calib_targets_pct_df[cols].sum(axis=0), axis=1)

        #trip_mc_calib_targets_pct_df.loc['All'] = trip_mc_calib_targets_pct_df.sum()
        trip_mc_calib_targets_pct_df.loc['All'] = trip_mc_calib_targets_pct_df.sum(axis=0)
        
        trip_mc_calib_targets_pct_df.name = trip_mc_calib_targets_df.name
        
        trip_mc_calib_targets_dfs.append(trip_mc_calib_targets_df)
        hts_trip_mc_dfs_updated.append(hts_trip_tour_mode_df)
        trip_mc_calib_targets_pct_dfs.append(trip_mc_calib_targets_pct_df)

    return obs_trip_target_dfs, trip_mc_calib_targets_dfs, hts_trip_mc_dfs_updated, trip_mc_calib_targets_pct_dfs

In [62]:
hts_trip_mc_dfs_init = hts_trip_mc_dfs.copy()
obs_trip_target_dfs, trip_mc_calib_targets_dfs, hts_trip_mc_dfs, trip_mc_calib_targets_pct_dfs = create_trip_mode_choice_targets(
    hts_trip_mc_dfs_init, obs_tour_mc_trips_dfs)

1768583.0
955714.0000000001
             OBS_trips row_scaling_factor col_scaling_factor
DRIVEALONE        <NA>               <NA>               <NA>
SHARED2           <NA>               <NA>               <NA>
SHARED3           <NA>               <NA>               <NA>
WALK              <NA>               <NA>               <NA>
BIKE              <NA>               <NA>               <NA>
WALK-TRANSIT  955714.0           0.540384           0.540384
PNR-TRANSIT       <NA>               <NA>               <NA>
KNR-TRANSIT       <NA>               <NA>               <NA>
TNC-TRANSIT       <NA>               <NA>               <NA>
TAXI              <NA>               <NA>               <NA>
TNC-REG           <NA>               <NA>               <NA>
TNC-SHARED        <NA>               <NA>               <NA>
SCHOOLBUS         <NA>               <NA>               <NA>
All               <NA>               <NA>               <NA>
221014.0
189409.0
             OBS_trips row_scaling_fact

310923.0
83398.0
             OBS_trips row_scaling_factor col_scaling_factor
DRIVEALONE        <NA>               <NA>               <NA>
SHARED2           <NA>               <NA>               <NA>
SHARED3           <NA>               <NA>               <NA>
WALK              <NA>               <NA>               <NA>
BIKE              <NA>               <NA>               <NA>
WALK-TRANSIT   83398.0           0.268227           0.268227
PNR-TRANSIT       <NA>               <NA>               <NA>
KNR-TRANSIT       <NA>               <NA>               <NA>
TNC-TRANSIT       <NA>               <NA>               <NA>
TAXI              <NA>               <NA>               <NA>
TNC-REG           <NA>               <NA>               <NA>
TNC-SHARED        <NA>               <NA>               <NA>
SCHOOLBUS         <NA>               <NA>               <NA>
All               <NA>               <NA>               <NA>
20445.0
3004.0
             OBS_trips row_scaling_factor col_scaling

C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\leah

In [63]:
# Creating a table that is the sum of all non-Total purposes for consistency check
summed_purposes_df = trip_mc_calib_targets_dfs[1].copy()
for i in range(2, len(trip_mc_calib_targets_dfs)):
    summed_purposes_df = summed_purposes_df + trip_mc_calib_targets_dfs[i]
summed_purposes_df.name = 'Non-Total Sum'
trip_mc_calib_targets_dfs.append(summed_purposes_df)

In [64]:
hts_trip_mc_dfs[0]


grouped_tour_mode,DRIVEALONE,SHARED2,SHARED3,WALK,BIKE,WALK-TRANSIT,PNR-TRANSIT,KNR-TRANSIT,TNC-TRANSIT,TAXI,TNC-REG,TNC-SHARED,SCHOOLBUS,All
grouped_linked_trip_mode,,,,,,,,,,,,,,
DRIVEALONE,13415263.0,2265664.0,906413.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16587341.0
SHARED2,0.0,5892131.0,1062605.0,0.0,0.0,0.0,0.0,0.0,0.0,23433.0,14162.0,7648.0,105825.0,7105804.0
SHARED3,0.0,0.0,4088340.0,0.0,0.0,0.0,0.0,0.0,0.0,7866.0,9395.0,1976.0,101340.0,4208916.0
WALK,171395.0,165892.0,140496.0,2633249.0,0.0,507369.0,0.0,0.0,0.0,6646.0,52909.0,18249.0,32468.0,3728673.0
BIKE,0.0,0.0,0.0,0.0,377237.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,377237.0
WALK-TRANSIT,0.0,0.0,0.0,0.0,0.0,1768583.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1768583.0
PNR-TRANSIT,0.0,0.0,0.0,0.0,0.0,0.0,221014.0,0.0,0.0,0.0,0.0,0.0,0.0,221014.0
KNR-TRANSIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,145621.0,0.0,0.0,0.0,0.0,0.0,145621.0
TNC-TRANSIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7224.0,0.0,0.0,0.0,0.0,7224.0


In [65]:
# writing HTS trip mode choice targets
write_tables_to_excel(
    dfs=hts_trip_mc_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=0,
    title='HTS Survey Trips',
    sep_for_col_title=5,
    col_title='Tour Mode'
)

# OBS trips and scaling factors
write_tables_to_excel(
    dfs=obs_trip_target_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=15,
    title='OBS Trip Targets',
    sep_for_col_title=1,
    col_title=''
)

# Trip Mode Targets
write_tables_to_excel(
    dfs=trip_mc_calib_targets_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=20,
    title='Trip Mode Choice Targets',
    sep_for_col_title=5,
    col_title='Tour Mode'
)

# Trip Mode Targets
write_tables_to_excel(
    dfs=trip_mc_calib_targets_pct_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=36,
    title='Trip Mode Choice Targets - Distribution',
    sep_for_col_title=5,
    col_title='Tour Mode'
)

In [66]:
excel_writer.save()
excel_writer.close()

C:\Users\leah.flake\AppData\Local\conda\conda\envs\cmap_abm\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## Writing Final Tables

In [67]:
print('Tour mode choice calibration targets written to: ', tour_mode_choice_output_path)
for df in tour_mc_calib_targets_dfs:
    table_name = df.name.replace('Purpose: ','') + '.csv'
    print(table_name)
    df.to_csv(os.path.join(tour_mode_choice_output_path, table_name))

print('Trip mode choice calibration targets written to: ', trip_mode_choice_output_path)
for df in trip_mc_calib_targets_dfs:
    table_name = df.name.replace('Purpose: ','') + '.csv'
    print(table_name)
    df.to_csv(os.path.join(trip_mode_choice_output_path, table_name))

Tour mode choice calibration targets written to:  output\tour_mode_choice
Total.csv
Work.csv
University.csv
School.csv
Work sub-tour.csv
Ind-Maintenance.csv
Ind-Discretionary.csv
Joint-Maintenance.csv
Joint-Discretionary.csv
Non-Total Sum.csv
Trip mode choice calibration targets written to:  output\trip_mode_choice
Total.csv
Work.csv
University.csv
School.csv
Work sub-tour.csv
Ind-Maintenance.csv
Ind-Discretionary.csv
Joint-Maintenance.csv
Joint-Discretionary.csv
Non-Total Sum.csv


In [68]:
# Merging final Tables into one output table for the automated calibration script
def melt_df(df, melt_id_var, value_name):
    melted_df = df.reset_index().melt(id_vars=[melt_id_var])
    melted_df.rename(columns={'value': value_name}, inplace=True)
    melted_df['purpose'] = df.name.replace('Purpose: ', '')
    return melted_df

melted_dfs = []
# Don't want to include the non-total sum table
for df in tour_mc_calib_targets_dfs[:-1]:
    melted_df = melt_df(df, melt_id_var='grouped_tour_mode', value_name='tours')
    melted_dfs.append(melted_df)
tour_mode_choice_calibration_table = pd.concat(melted_dfs)

melted_dfs = []
# Don't want to include the non-total sum table
for df in trip_mc_calib_targets_dfs[:-1]:
    melted_df = melt_df(df, melt_id_var='grouped_linked_trip_mode', value_name='trips')
    melted_dfs.append(melted_df)
trip_mode_choice_calibration_table = pd.concat(melted_dfs)

tour_mode_choice_calibration_table.to_csv(os.path.join(output_path, 'tour_mode_choice_calibration_targets.csv'), index=False)
trip_mode_choice_calibration_table.to_csv(os.path.join(output_path, 'trip_mode_choice_calibration_targets.csv'), index=False)

## Output Statistics

In [69]:
trip_counts_df = pd.DataFrame(index=['WALK-TRANSIT', 'PNR-TRANSIT', 'KNR-TRANSIT'])
trip_counts_df['HTS Raw'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'HTS Raw'] = len(spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT'])
trip_counts_df.loc['PNR-TRANSIT', 'HTS Raw'] = len(spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT'])
trip_counts_df.loc['KNR-TRANSIT', 'HTS Raw'] = len(spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT'])
trip_counts_df.loc['TNC-TRANSIT', 'HTS Raw'] = len(spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'TNC-TRANSIT'])

trip_counts_df['HTS Weighted'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'HTS Weighted'] = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT']['PER_WEIGHT'].sum()
trip_counts_df.loc['PNR-TRANSIT', 'HTS Weighted'] = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT']['PER_WEIGHT'].sum()
trip_counts_df.loc['KNR-TRANSIT', 'HTS Weighted'] = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT']['PER_WEIGHT'].sum()
trip_counts_df.loc['TNC-TRANSIT', 'HTS Weighted'] = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'TNC-TRANSIT']['PER_WEIGHT'].sum()

trip_counts_df['OBS Raw'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'OBS Raw'] = len(obs_df[obs_df['grouped_tour_mode'] == 'WALK-TRANSIT'])
trip_counts_df.loc['PNR-TRANSIT', 'OBS Raw'] = len(obs_df[obs_df['grouped_tour_mode'] == 'PNR-TRANSIT'])
trip_counts_df.loc['KNR-TRANSIT', 'OBS Raw'] = len(obs_df[obs_df['grouped_tour_mode'] == 'KNR-TRANSIT'])
trip_counts_df.loc['TNC-TRANSIT', 'OBS Raw'] = len(obs_df[obs_df['grouped_tour_mode'] == 'TNC-TRANSIT'])

trip_counts_df['OBS Weighted'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'OBS Weighted'] = obs_df[obs_df['grouped_tour_mode'] == 'WALK-TRANSIT']['linked_weight_scaled'].sum()
trip_counts_df.loc['PNR-TRANSIT', 'OBS Weighted'] = obs_df[obs_df['grouped_tour_mode'] == 'PNR-TRANSIT']['linked_weight_scaled'].sum()
trip_counts_df.loc['KNR-TRANSIT', 'OBS Weighted'] = obs_df[obs_df['grouped_tour_mode'] == 'KNR-TRANSIT']['linked_weight_scaled'].sum()
trip_counts_df.loc['TNC-TRANSIT', 'OBS Weighted'] = obs_df[obs_df['grouped_tour_mode'] == 'TNC-TRANSIT']['linked_weight_scaled'].sum()

trip_counts_df.loc['All'] = trip_counts_df.sum()
print('Trips in each survey:')
trip_counts_df

Trips in each survey:


,HTS Raw,HTS Weighted,OBS Raw,OBS Weighted
WALK-TRANSIT,8605,1768582.735905,90029,955714.374657
PNR-TRANSIT,1349,221014.196068,68976,189409.346779
KNR-TRANSIT,634,145621.486899,21253,90509.359629
TNC-TRANSIT,38,7223.819611,2639,15413.183461
All,10626,2142442.238483,182897,1251046.264525


In [70]:

    
# creating output tables for visualizer
mode_vis_dict = {
    'DRIVEALONE': 1,
    'SHARED2': 2,
    'SHARED3': 3,
    'WALK': 4,
    'BIKE': 5,
    'WALK-TRANSIT': 6,
    'PNR-TRANSIT': 7,
    'KNR-TRANSIT': 8,
    'TNR-TRANSIT': 9,
    'TNC-TRANSIT': 9,
    'SCHOOLBUS': 10,
    'TAXI': 11,
    'All': 'Total'
}
purpose_vis_dict = {
    'University': 'univ',
    'School': 'sch',
    'Work': 'work',
    'Work sub-tour': 'atwork',
    'Ind-Discretionary': 'idisc',
    'Ind-Maintenance': 'imain',
    'Joint-Maintenance': 'jmain',
    'Joint-Discretionary': 'jdisc',
    'Total': 'Total'
}



tour_mc_vis = tour_mode_choice_calibration_table.copy()
tour_mc_vis.loc[tour_mc_vis['grouped_tour_mode'].isin(['TNC-REG','TNC-SHARED']), 'grouped_tour_mode'] = 'TAXI'
tour_mc_vis = tour_mc_vis[tour_mc_vis['grouped_tour_mode'] != 'All']
tour_mc_vis['id'] = tour_mc_vis['grouped_tour_mode'].apply(lambda x: mode_vis_dict[x])
tour_mc_vis['purpose'] = tour_mc_vis['purpose'].apply(lambda x: purpose_vis_dict[x])
tour_mc_vis = tour_mc_vis.reset_index(drop=True).pivot_table(
    columns='auto_suff', values='tours', index=['id','purpose']).reset_index()
tour_mc_vis = tour_mc_vis.rename(columns={0: 'freq_as0', 1: 'freq_as1', 2: 'freq_as2', 'All': 'freq_all'})
tour_mc_vis.to_csv(os.path.join(output_path,'visualizer_summaries','tmodeProfile_vis_calib.csv'), index=False)



In [71]:


modename_vis_dict = {
    'DRIVEALONE': 'Auto SOV',
    'SHARED2': 'Auto 2 Person',
    'SHARED3': 'Auto 3+ Person',
    'WALK': 'Auto SOV',
    'BIKE': 'Bike/Moped',
    'WALK-TRANSIT': 'Walk-Transit',
    'PNR-TRANSIT': 'PNR-Transit',
    'KNR-TRANSIT': 'KNR-Transit',
    'TNC-TRANSIT': 'TNR-Transit',
    'TAXI': 'Taxi',
    'SCHOOLBUS': 'School Bus',
    'All': 'Total'
}
trip_purpose_vis_dict = purpose_vis_dict
trip_purpose_vis_dict['Total'] = 'total'
trip_purpose_vis_dict['School'] = 'schl'
trip_mc_vis = trip_mode_choice_calibration_table.copy()

trip_mc_vis.loc[trip_mc_vis['grouped_linked_trip_mode'].isin(['TNC-REG','TNC-SHARED']), 'grouped_linked_trip_mode'] = 'TAXI'
trip_mc_vis.loc[trip_mc_vis['grouped_tour_mode'].isin(['TNC-REG','TNC-SHARED']), 'grouped_tour_mode'] = 'TAXI'

trip_mc_vis = trip_mc_vis.groupby(['grouped_linked_trip_mode','grouped_tour_mode', 'purpose']).trips.sum().reset_index()


trip_mc_vis = trip_mc_vis[trip_mc_vis['grouped_linked_trip_mode'] != 'All']
trip_mc_vis['tripmode'] = trip_mc_vis['grouped_linked_trip_mode'].apply(lambda x: mode_vis_dict[x])
trip_mc_vis['tourmode'] = trip_mc_vis['grouped_tour_mode'].apply(lambda x: modename_vis_dict[x])

trip_mc_vis['value'] = trip_mc_vis['trips']
trip_mc_vis.loc[trip_mc_vis['tourmode'] == 'All', 'tourmode'] = 'Total'
trip_mc_vis['tourmode_num'] = trip_mc_vis['grouped_tour_mode'].apply(lambda x: mode_vis_dict[x])
trip_mc_vis['purpose'] = trip_mc_vis['purpose'].apply(lambda x: trip_purpose_vis_dict[x])
trip_mc_vis['grp_var'] = trip_mc_vis.apply(lambda row: \
                                           row['purpose'] + 'tourmode' + str(row['tourmode_num'])
                                           if row['tourmode'] != 'Total' \
                                           else  row['purpose'] + str(row['tourmode_num']), axis=1)
trip_mc_vis_cols = ['tripmode', 'tourmode', 'purpose', 'value', 'grp_var']
trip_mc_vis[trip_mc_vis_cols].to_csv(os.path.join(output_path,'visualizer_summaries','tripModeProfile_vis_calib.csv'), index=False)
